In [ ]:
# Install cell, Install if needed

#pip install sklearn
#pip install Keras-Preprocessing

In [1]:
#import block

import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from typing import Union
import matplotlib.pyplot as plt
from keras_flops import get_flops
import pandas as pd
#ResNet Resource
#https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b
import tensorflow.keras as K

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from keras_preprocessing.image import ImageDataGenerator
import numpy as np
import os
from PIL import Image
from skimage import io

from keras.models import Model

tf.enable_v2_behavior()

2022-10-29 21:20:51.215724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load Data

In [63]:
#load data use panda

#read annotation file
annotations = pd.read_csv("./annotations.csv", skiprows = 1,
                         names = ["Image Name","Majority Vote Label","Number of Annotations who Selected SSA(Out of 7)","Partition"])
annotations.head()

#Create dataframe
df = pd.DataFrame(annotations,
                 columns = ["Image Name","Majority Vote Label","Number of Annotations who Selected SSA(Out of 7)", "Partition"])
df.set_index("Image Name",inplace = True)

df.head()

,Majority Vote Label,Number of Annotations who Selected SSA(Out of 7),Partition
Image Name,,,
MHIST_aaa.png,SSA,6,train
MHIST_aab.png,HP,0,train
MHIST_aac.png,SSA,5,train
MHIST_aae.png,HP,1,train
MHIST_aaf.png,SSA,5,train


# Data Augmentation

In [62]:
#resource: https://www.youtube.com/watch?v=ccdssX4rIh8
#may be need to change later
#right now all the images are saved to local train directory, we may be
#able to directly flow the all the images and store them in the memory

#the image prefix is the label of the image

def create_images(partition,label):
    SIZE = 224
    image_directory = "images/"
    data_set = []
    
    datagen = ImageDataGenerator(
        rotation_range = 180,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.5,
        horizontal_flip = True,
        fill_mode = 'nearest'
    )
    
    my_images = os.listdir(image_directory)
    length = 0
    for i,image_name in enumerate(my_images):
        if df.loc[image_name][2] == partition and df.loc[image_name][0] == label:
            image = io.imread(image_directory+image_name)
            image = Image.fromarray(image,'RGB')
            image = image.resize((SIZE,SIZE))
            data_set.append(np.array(image))
            length = length + 1
    x = np.array(data_set)
    i = 0
    for batch in datagen.flow(x,
                              batch_size = 1,
                              save_to_dir = partition,
                              save_prefix = label,
                              save_format = 'png'):
        i = i + 1
        #change length to change the number of images generates
        if i < length:
            continue
        else:
            break

#create directory before run the code
create_images('train','SSA')
create_images('train','HP')

630
360


# Create Model

In [ ]:
#teacher model

def teacher_model():
    res_model = K.applications.resnet_v2.ResNet50V2(include_top = False, weights="imagenet",input_shape=(224,224,3))
    for layers in res_model.layers:
        layer.trainable = False
    x = K.layers.Flatten()(res_model.output)
    outputs = K.layers.Dense(10,activation="softmax")(x) 
    teacher = Model(input = resnet_model.input, outputs = outputs)
    return teacher

#student model
def student_model():
    mob_model = K.applications.mobilenet_v2.MobileNetV2(include_top = False, weights="imagenet", input_shape=(224,224,3))
    for layers in mob_model.layers:
        layer.trainable = False
    x = K.layers.Flatten()(mob_model.output)
    outputs = K.layers.Dense(10, activation = "softmax")(x)
    student = Model(input = mob_model.input, outputs = outputs)
    return student

In [ ]:
#build Distiller
#Source: https://keras.io/examples/vision/knowledge_distillation/

class Distiller(keras.Model):
    def __init__(self,student,teacher):
        super(Distiller,self).__init__()
        self.teacher = teacher
        self.student = student
        
    def compile(self,
                optimizer,
                metrics,
                student_loss_fn,
                distillation_loss_fn,
                alpha = 0.5,
                temperature = 4):
        super(Distiller,self).compile(optimizer = optimizer, metrics = metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_fn
        self.alpha = alpha
        self.temperature = temperature
    
    def train_setp(self, data):
        x,y = data
        
        #Forward pass of teacher
        teacher_predictions = self.teacher(x, training = False)
        
        #compute losses
        student_loss = self.student_loss_fn(y, student_predictions)
        distillation_loss = self.distillation_loss_fn(
            tf.nn.softmax(teacher_predictions/self.temperature, axis = 1)
            tf.nn.softmax(student_predicitons/self.temperature, axis = 1)
        )
        loss = self.alpha * student_loss + (1-self.alpha) * distillation_loss
        
        #compute gradient
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss,trainable_vars)
        #update weights
        self.optimizer.apply_gradients(zip(gradients, trainables_vars))
        #update the metrics configured in compile()
        self.compiled_metrics.update_state(y, student_predictions)
        #Return a dict of performance
        results = {m.name:m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}=
        )
        return results
    
    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results